In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

*Author:Razin Karimi*<br>
Learnings: UNet, Activaton functions, Segmentation


In [ ]:
from tensorflow.keras import models,layers,datasets

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from keras import metrics
import keras
import os
import random
from tqdm import tqdm 
from zipfile import ZipFile
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class UNet:
    def __init__(self,input_shape_,activation_="relu"):
        self.input_shape=input_shape_
        self.activation=activation_
        
    def dice_coef(self,y_true,y_pred,smooth=100):
        y_true=tf.cast(y_true,tf.float32)
        y_pred=tf.cast(y_pred,tf.float32)
        
        y_true_f=K.flatten(y_true)
        y_pred_f=K.flatten(y_pred)
        
        intersection=K.sum(y_true_f*y_pred_f)
        
        dice=(2. *intersection+smooth)/(K.sum(y_true_f)+K.sum(y_pred_f)+smooth)
        return dice
    
    def dice_coef_loss(self,y_true,y_pred):
        return -self.dice_coef(y_true,y_pred)
    
    def iou(self,y_true,y_pred):
        y_true=tf.cast(y_true,tf.float32)
        y_pred=tf.cast(y_pred,tf.float32)
        
        def f(y_true,y_pred):
            intersection=(y_true*y_pred).sum()
            union=y_true.sum()+y_pred.sum()-intersection
            x=(intersection+1e-15)/(union+1e-15)
            x=x.astype(np.float32)
            return x
        
        return tf.numpy_function(f,[y_true,y_pred],tf.float32)
    
    def buildModel(self):

        kernel_initializer='he_uniform'
        inputs=tf.keras.Input(shape=self.input_shape)
        s=inputs
        #contraction
        c1=Conv2D(16,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(s)
        c1=Conv2D(16,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c1)
        m1=MaxPooling2D(2,2)(c1)
        m1=Dropout(0.25)(m1)
        
        c2=Conv2D(32,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(m1)
        c2=Conv2D(32,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c2)
        m2=MaxPooling2D(2,2)(c2)
        m2=Dropout(0.25)(m2)
        
        c3=Conv2D(64,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(m2)
        c3=Conv2D(64,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c3)
        m3=MaxPooling2D(2,2)(c3)
        m3=Dropout(0.25)(m3)
        
        
        c4=Conv2D(128,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(m3)
        c4=Conv2D(128,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c4)
        m4=MaxPooling2D(2,2)(c4)
        m4=Dropout(0.25)(m4)
        
        c5=Conv2D(256,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(m4)
        c5=Conv2D(256,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c5)
        m5=MaxPooling2D(2,2)(c5)
        m5=Dropout(0.25)(m5)
        
        #middle layer
        c6=Conv2D(512,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(m5)
        c6=Conv2D(512,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c6)
        
        
        #decontraction
        
        dc1=Conv2DTranspose(256,(2, 2), strides=(2, 2),padding='same')(c6)
        dc1=concatenate([dc1,c5])
        dc1=Dropout(0.25)(dc1)
        c7=Conv2D(256,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(dc1)
        c7=Conv2D(256,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c7)
        
        dc2=Conv2DTranspose(128,(2, 2), strides=(2, 2),padding='same')(c7)
        dc2=concatenate([dc2,c4])
        dc2=Dropout(0.25)(dc2)
        c8=Conv2D(128,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(dc2)
        c8=Conv2D(128,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c8)
        
        dc3=Conv2DTranspose(64,(2, 2), strides=(2, 2),padding='same')(c8)
        dc3=concatenate([dc3,c3])
        dc3=Dropout(0.25)(dc3)
        c9=Conv2D(64,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(dc3)
        c9=Conv2D(64,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c9)
        
        dc4=Conv2DTranspose(32,(2, 2), strides=(2, 2),padding='same')(c9)
        dc4=concatenate([dc4,c2])
        dc4=Dropout(0.25)(dc4)
        c10=Conv2D(32,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(dc4)
        c10=Conv2D(32,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c10)
        
        dc5=Conv2DTranspose(16,(2, 2), strides=(2, 2),padding='same')(c10)
        dc5=concatenate([dc5,c1])
        dc5=Dropout(0.25)(dc5)
        c11=Conv2D(16,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(dc5)
        c11=Conv2D(16,(3,3),activation=self.activation,kernel_initializer=kernel_initializer,padding='same')(c11)
        
        
        outputs=Conv2D(1,(1,1),activation="sigmoid")(c11)
        
        model=Model(inputs=[inputs],outputs=[outputs])
        model.summary()
        
        return model
    
    def CompileandSummarize(self,model_):
        model_.compile(optimizer=tf.keras.optimizers.Adam(),loss=self.dice_coef_loss,metrics=self.iou)
        
        
        
        
        
    
        

In [ ]:
class MexicanReLU(tf.keras.layers.Layer):
    def __init__(self, alpha=1.0, sigma=1.0, mu=0.0):
        super(MexicanReLU, self).__init__()
        self.alpha = tf.Variable(alpha, trainable=True)
        self.sigma = tf.Variable(sigma, trainable=True)
        self.mu = tf.Variable(mu, trainable=True)

    def call(self, inputs):
        relu = tf.nn.relu(inputs)
        gaussian = self.alpha * tf.exp(-tf.square(inputs - self.mu) / (2 * tf.square(self.sigma))) * (inputs - self.mu)
        return relu + gaussian

In [ ]:
class Swish(tf.keras.layers.Layer):
    def __init__(self):
        super(Swish, self).__init__()

    def call(self, inputs):
        return inputs * tf.keras.backend.sigmoid(inputs)

In [ ]:
class TrainableLeakyReLU(tf.keras.layers.Layer):
    def __init__(self, alpha_initializer=tf.initializers.constant(0.2), **kwargs):
        super(TrainableLeakyReLU, self).__init__(**kwargs)
        self.alpha_initializer = tf.keras.initializers.get(alpha_initializer)
    
    def build(self, input_shape):
        self.alpha = self.add_weight(shape=(), initializer=self.alpha_initializer, trainable=True, name="alpha")
        super(TrainableLeakyReLU, self).build(input_shape)
    
    def call(self, inputs):
        return tf.maximum(inputs, self.alpha * inputs)
    
    def get_config(self):
        config = super(TrainableLeakyReLU, self).get_config()
        config.update({"alpha_initializer": self.alpha_initializer})
        return config

In [ ]:
class ISRU(tf.keras.layers.Layer):
    def __init__(self, alpha=1.0, **kwargs):
        super(ISRU, self).__init__(**kwargs)
        self.alpha = tf.Variable(initial_value=alpha, trainable=True, name='alpha')

    def call(self, inputs):
        return inputs / tf.sqrt(1.0 + self.alpha * tf.square(inputs))

In [ ]:
class Mish(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)

    def call(self, inputs):
        return inputs * tf.math.tanh(tf.math.softplus(inputs))

In [ ]:
class gated_swish(tf.keras.layers.Layer):
    
    def __init__(self):
        super(gated_swish, self).__init__()

    def call(self, inputs):

        """
        Gated Swish activation function.
        """
        gate = tf.math.sigmoid(inputs)
        activation = tf.nn.swish(inputs)
        
        return gate * activation

In [ ]:
class SeLU(tf.keras.layers.Layer):
    def __init__(self, alpha=1.6732632423543772848170429916717, scale=1.0507009873554804934193349852946, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha
        self.scale = scale

    def call(self, inputs):
        return self.scale * tf.where(inputs > 0.0, inputs, self.alpha * tf.exp(inputs) - self.alpha)

    def get_config(self):
        config = super().get_config()
        config.update({'alpha': self.alpha, 'scale': self.scale})
        return config

In [ ]:
import tensorflow as tf

class LearnableSwish(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(LearnableSwish, self).__init__(**kwargs)

    def build(self, input_shape):
        self.beta = self.add_weight(shape=(1, ), initializer="ones", trainable=True)
        super(LearnableSwish, self).build(input_shape)

    def call(self, inputs):
        return tf.nn.swish(inputs * self.beta)

In [ ]:
class SRS(tf.keras.layers.Layer):
    def __init__(self, alpha=1.0, **kwargs):
        super(SRS, self).__init__(**kwargs)
        self.alpha = alpha

    def call(self, inputs):
        return tf.maximum(inputs, self.alpha * (tf.sigmoid(inputs) - 0.5))

In [ ]:
class TanELU(tf.keras.layers.Layer):
    def __init__(self, alpha=1.0, **kwargs):
        super(TanELU, self).__init__(**kwargs)
        self.alpha = alpha

    def call(self, inputs):
        pos = tf.nn.relu(inputs)
        neg = self.alpha * (tf.exp(inputs) - 1)
        return tf.where(inputs > 0, tf.tanh(pos), tf.tanh(neg))

In [ ]:
from timeit import default_timer as timer
class TimingCallback(keras.callbacks.Callback):
    def __init__(self,logs={}):
        self.logs=[]
        
        
    def on_epoch_begin(self,epoch,logs={}):
        self.starttime=timer()
        
    def on_epoch_end(self,epoch,logs={}):
        self.logs.append(timer()-self.starttime)
        
        
    
    


In [ ]:
with ZipFile("../input/data-science-bowl-2018/stage1_train.zip","r") as zipref:
    zipref.extractall("./stage1_train")

with ZipFile("../input/data-science-bowl-2018/stage1_test.zip","r") as zipref:
    zipref.extractall("./stage1_test")

In [ ]:
seed=42
np.random.seed=seed

Image_height=96
Image_width=96
Image_channels=3

Train_path='./stage1_train/'
Test_path='./stage1_test/'

In [ ]:
train_ids=next(os.walk(Train_path))[1]
test_ids=next(os.walk(Test_path))[1]
train_ids[:5]

In [ ]:
X_train=np.zeros((len(train_ids),Image_height,Image_width,Image_channels),dtype=np.uint8)
y_train=np.zeros((len(train_ids),Image_height,Image_width,1),dtype=np.bool)
print('Resizing images and masks')

for n,filename in tqdm(enumerate(train_ids)):
    path=Train_path+filename
    img=imread(path+'/images/'+filename+'.png')[:,:,:Image_channels]
    img=resize(img,(Image_height,Image_width),mode='constant',preserve_range=True)
    X_train[n]=img
    mask=np.zeros((Image_height,Image_width,1),dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (Image_height,Image_width), mode='constant',  
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)  
            
    y_train[n] = mask




In [ ]:
X_test = np.zeros((len(test_ids),Image_height , Image_width, Image_channels), dtype=np.uint8)
sizes_test = []
print('Resizing test images') 
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = Test_path + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:Image_channels
                                                  ]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (Image_height, Image_width), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [ ]:
imshow(X_train[0])
plt.show()
imshow(y_train[0])
plt.show()
imshow(np.squeeze(y_train[0]))
plt.show()

In [ ]:
X_test,y_test=X_train[536:],y_train[536:]
X_train,y_train=X_train[0:536],y_train[0:536]


In [ ]:
(X_train.shape,y_train.shape),(X_test.shape,y_test.shape)


In [ ]:
!mkdir /kaggle/working/logs



In [ ]:
tb_1 = tf.keras.callbacks.TensorBoard(log_dir='/kaggle/working/logs', profile_batch=(5, 8))


In [ ]:
class GCU(tf.keras.layers.Layer):
    def __init__(self):
        super(GCU, self).__init__()
    
    def call(self, x):
        return tf.cast(tf.math.multiply(x , tf.math.cos(x))/tf.reduce_sum(x), tf.float32)

In [ ]:
training_inferences={}


In [ ]:
cb = TimingCallback()

UNet_1 = UNet((96, 96, 3), activation_= MexicanReLU())
model1 = UNet_1.buildModel()
UNet_1.CompileandSummarize(model1)
results1 = model1.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks = [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],  validation_data = (X_test, y_test))

training_inferences['MexicanReLU'] = sum(cb.logs)

In [ ]:
results1.history.keys()



In [ ]:
plt.plot(results1.history['loss'])

In [ ]:
plt.plot(results1.history['iou'])

In [ ]:
cb = TimingCallback()

UNet_2 = UNet((96, 96, 3), activation_= Swish())
model2 = UNet_2.buildModel()
UNet_2.CompileandSummarize(model2)
results2 = model2.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks =  [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],  validation_data = (X_test, y_test))

training_inferences['Swish'] = sum(cb.logs)

In [ ]:
plt.plot(results2.history['loss'])

In [ ]:
plt.plot(results2.history['iou'])

In [ ]:
cb = TimingCallback()

UNet_3 = UNet((96, 96, 3), activation_= TrainableLeakyReLU())
model3 = UNet_3.buildModel()
UNet_3.CompileandSummarize(model3)
results3 = model3.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks =  [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)],  validation_data = (X_test, y_test))

training_inferences['TrainableLeakyReLU'] = sum(cb.logs)

In [ ]:
plt.plot(results3.history['loss'])

In [ ]:
plt.plot(results3.history['iou'])

In [ ]:
training_inferences

In [ ]:
cb = TimingCallback()

UNet_4 = UNet((96, 96, 3), activation_= ISRU())
model4 = UNet_4.buildModel()
UNet_4.CompileandSummarize(model4)
results4 = model4.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks = [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['ISRU'] = sum(cb.logs)

In [ ]:
plt.plot(results4.history['loss'])

In [ ]:
plt.plot(results4.history['iou'])

In [ ]:
cb = TimingCallback()

UNet_5 = UNet((96, 96, 3), activation_= Mish())
model5 = UNet_5.buildModel()
UNet_5.CompileandSummarize(model5)
results5 = model5.fit(x = X_train, y = y_train, batch_size = 8, epochs=150,callbacks= [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['Mish'] = sum(cb.logs)

In [ ]:
plt.plot(results5.history['loss'])

In [ ]:
plt.plot(results5.history['iou'])

In [ ]:
cb = TimingCallback()

UNet_6 = UNet((96, 96, 3), activation_= gated_swish())
model6 = UNet_6.buildModel()
UNet_6.CompileandSummarize(model6)
results6 = model6.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks =  [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['gated_swish'] = sum(cb.logs)

In [ ]:
plt.plot(results6.history['loss'])

In [ ]:
plt.plot(results6.history['iou'])

In [ ]:
cb = TimingCallback()

UNet__ = UNet((96, 96, 3), activation_= "relu")
model_ = UNet__.buildModel()
UNet__.CompileandSummarize(model_)
results_ = model5.fit(x = X_train, y = y_train, batch_size = 8, epochs=150,callbacks = [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['relu'] = sum(cb.logs)

In [ ]:
plt.plot(results_.history['loss'])

In [ ]:
plt.plot(results_.history['iou'])

In [ ]:
cb = TimingCallback()

UNet_ls = UNet((96, 96, 3), activation_= LearnableSwish())
modells = UNet_ls.buildModel()
UNet_ls.CompileandSummarize(modells)
resultsls = modells.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks =  [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['LearnableSwish'] = sum(cb.logs)

In [ ]:
plt.plot(resultsls.history['loss'])

In [ ]:
cb = TimingCallback()

UNet_srs = UNet((96, 96, 3), activation_= SRS())
modelsrs = UNet_srs.buildModel()
UNet_srs.CompileandSummarize(modelsrs)
resultsrs = modelsrs.fit(x = X_train, y = y_train, batch_size = 8, epochs=150, callbacks =  [cb, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)], validation_data = (X_test, y_test))

training_inferences['SRS'] = sum(cb.logs)

In [ ]:
plt.plot(resultsrs.history['loss'])

In [ ]:
x = []
y = []

for i in training_inferences:
    x.append(i)
    y.append(training_inferences[i])

In [ ]:
fig, ax = plt.subplots()
plt.bar(range(len(y)), y, tick_label=x)
plt.xticks(rotation='vertical')
fig.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Create a figure with 2 rows and 4 columns of subplots
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(12, 8))

# Plot the data on the subplots
axs[0, 0].plot(results1.history['loss'])
axs[0, 1].plot(results2.history['loss'])
axs[0, 2].plot(results3.history['loss'])
axs[1, 0].plot(results4.history['loss'])
axs[1, 1].plot(results5.history['loss'])
axs[1, 2].plot(results6.history['loss'])
axs[2, 0].plot(results_.history['loss'])
axs[2, 1].plot(resultsls.history['loss'])
axs[2,2].plot(resultsrs.history['loss'])

# Add titles to the subplots
axs[0, 0].set_title('Symmetric_Galu')
axs[0, 1].set_title('Swish')
axs[0, 2].set_title('TrainableLeakyReLU')
axs[1, 0].set_title('ISRU')
axs[1, 1].set_title('Mish')
axs[1, 2].set_title('Gated_Swish')
axs[2, 0].set_title('ReLu')
axs[2, 1].set_title('LearnableSwish')
axs[2, 2].set_title('SRS')

# Add x and y axis labels
for ax in axs.flat:
    ax.set(xlabel='Epoch', ylabel='Loss')

# Add a main title to the figure
fig.suptitle('8 Subplots')

# Adjust the spacing between subplots
fig.tight_layout()

# Display the figure
plt.show()


In [ ]:
batch_size = X_test.shape[0]  # Get the batch size
height, width, channels = 96, 96, 3  # Set the height, width, and channels

# Reshape the input tensor to match the expected shape
X_test_reshaped = np.reshape(X_test, (batch_size, height, width, channels))

In [ ]:

#Mexican_Relu
y_pred = model1.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred, axis=1)
import matplotlib.pyplot as plt

# Assuming you have already made predictions on your test data and you have the predicted masks in a variable y_pred

# Choose a random image from the test set to plot
index = 10 # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#Swish
y_pred2=model2.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred2, axis=1)
# Assuming you have already made predictions on your test data and you have the predicted masks in a variable y_pred

# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred2[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#TrainableLekyRelu
y_pred3=model3.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred3, axis=1)
# Assuming you have already made predictions on your test data and you have the predicted masks in a variable y_pred

# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred3[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#ISRU
y_pred4=model4.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred4, axis=1)
# Assuming you have already made predictions on your test data and you have the predicted masks in a variable y_pred

# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred4[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#Mish
y_pred5=model5.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred5, axis=1)
# Assuming you have already made predictions on your test data and you have the predicted masks in a variable y_pred

# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred5[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#Gated Swish
y_pred6=model6.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred6, axis=1)
# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred6[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#relu
y_pred7=model_.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred7, axis=1)
# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred7[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#Learnable Swish
y_pred8=modells.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred8, axis=1)
# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred8[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()

In [ ]:
#SRS
y_pred9=modelsrs.predict(X_test_reshaped)
predicted_classes = np.argmax(y_pred9, axis=1)
# Choose a random image from the test set to plot
index = 10  # Change this to the index of the image you want to plot
image = X_test[index]  # Get the original image data
mask_true = y_test[index]  # Get the true mask for the image
mask_pred = y_pred9[index]  # Get the predicted mask for the image

# Create a subplot with three columns to plot the original image, true mask and predicted mask side by side
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot the original image in the first column
ax[0].imshow(image)
ax[0].set_title("Original Image")

# Plot the true mask in the second column
ax[1].imshow(mask_true, cmap='gray')
ax[1].set_title("True Mask")

# Plot the predicted mask in the third column
ax[2].imshow(mask_pred, cmap='gray')
ax[2].set_title("Predicted Mask")

# Show the plot
plt.show()